In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('eventlog_large.csv', sep=';')
df.head()

,Case_ID,Activity,User,Timestamp
0,1,a,u2,2016-04-15 08:41:28
1,1,b,u3,2016-04-18 12:55:01
2,2,a,u2,2016-04-18 20:40:14
3,1,d,u5,2016-04-19 07:22:59
4,2,b,u4,2016-04-21 22:42:39


In [3]:
df_sort = df.copy()
df_sort = df_sort.sort_values(by=['Case_ID', 'Timestamp'], ascending=True)
df_sort.head()

,Case_ID,Activity,User,Timestamp
0,1,a,u2,2016-04-15 08:41:28
1,1,b,u3,2016-04-18 12:55:01
3,1,d,u5,2016-04-19 07:22:59
5,1,e,u7,2016-04-23 15:06:58
7,1,f,u7,2016-04-24 19:18:32


In [4]:
df_sort['Count'] = df_sort.groupby('Case_ID').cumcount()+1
df_sort

,Case_ID,Activity,User,Timestamp,Count
0,1,a,u2,2016-04-15 08:41:28,1
1,1,b,u3,2016-04-18 12:55:01,2
3,1,d,u5,2016-04-19 07:22:59,3
5,1,e,u7,2016-04-23 15:06:58,4
7,1,f,u7,2016-04-24 19:18:32,5
8,1,g,u5,2016-04-25 14:56:42,6
10,1,h,u2,2016-04-26 10:00:36,7
2,2,a,u2,2016-04-18 20:40:14,1
4,2,b,u4,2016-04-21 22:42:39,2
6,2,d,u6,2016-04-24 01:29:27,3


In [5]:
df_pivot = df_sort.copy()
df_pivot = df_pivot.pivot(index='Case_ID', columns='Count', values='User')
df_pivot

Count,1,2,3,4,5,6,7
Case_ID,,,,,,,
1,u2,u3,u5,u7,u7,u5,u2
2,u2,u4,u6,u5,u7,u7,u1
3,u2,u3,u5,u5,u8,u8,u2
4,u1,u3,u5,u5,u7,u8,u1
5,u1,u3,u2,NaN,NaN,NaN,NaN
6,u2,u4,u5,u6,u7,u7,u1
7,u1,u3,u1,NaN,NaN,NaN,NaN
8,u2,u4,u6,u5,u7,u7,u1
9,u1,u3,u5,u5,u7,u8,u1


In [6]:
df_pivot = df_pivot.fillna('X')
df_pivot = df_pivot.astype('str')
df_pivot['Trace'] = df_pivot.apply(lambda x: ','.join(x), axis=1)
df_pivot['Trace'] = df_pivot['Trace'].apply(lambda x: x.replace(',X',''))

df_pivot

Count,1,2,3,4,5,6,7,Trace
Case_ID,,,,,,,,
1,u2,u3,u5,u7,u7,u5,u2,"u2,u3,u5,u7,u7,u5,u2"
2,u2,u4,u6,u5,u7,u7,u1,"u2,u4,u6,u5,u7,u7,u1"
3,u2,u3,u5,u5,u8,u8,u2,"u2,u3,u5,u5,u8,u8,u2"
4,u1,u3,u5,u5,u7,u8,u1,"u1,u3,u5,u5,u7,u8,u1"
5,u1,u3,u2,X,X,X,X,"u1,u3,u2"
6,u2,u4,u5,u6,u7,u7,u1,"u2,u4,u5,u6,u7,u7,u1"
7,u1,u3,u1,X,X,X,X,"u1,u3,u1"
8,u2,u4,u6,u5,u7,u7,u1,"u2,u4,u6,u5,u7,u7,u1"
9,u1,u3,u5,u5,u7,u8,u1,"u1,u3,u5,u5,u7,u8,u1"


ModuleNotFoundError: No module named 'pygraphviz'

In [8]:
df_pivot_2=df_pivot[['Trace',1]].groupby(['Trace'], as_index=False).count()
df_pivot_2 = df_pivot_2.sort_values(by=1, ascending=False).reset_index()
del df_pivot_2['index']
df_pivot_2 = df_pivot_2.reset_index()
df_pivot_2['index'] = df_pivot_2['index']+1
df_pivot_2 = df_pivot_2.rename(columns={'index':'Variants',1:'Count'})
df_pivot_2

Count,Variants,Trace,Count
0,1,"u2,u3,u2",404
1,2,"u2,u3,u1",377
2,3,"u1,u3,u1",265
3,4,"u1,u3,u2",258
4,5,"u2,u4,u1",179
5,6,"u2,u4,u2",173
6,7,"u2,u3,u5,u6,u7,u8,u1",133
7,8,"u2,u3,u5,u6,u7,u7,u1",119
8,9,"u1,u4,u1",106
9,10,"u1,u4,u2",104


In [9]:
df_pivot_2.to_csv('Case_Count_2.csv', index=False)

In [13]:
#DF CASE
df_case = df_sort.copy()
df_case

,Case_ID,Activity,User,Timestamp,Count
0,1,a,u2,2016-04-15 08:41:28,1
1,1,b,u3,2016-04-18 12:55:01,2
3,1,d,u5,2016-04-19 07:22:59,3
5,1,e,u7,2016-04-23 15:06:58,4
7,1,f,u7,2016-04-24 19:18:32,5
8,1,g,u5,2016-04-25 14:56:42,6
10,1,h,u2,2016-04-26 10:00:36,7
2,2,a,u2,2016-04-18 20:40:14,1
4,2,b,u4,2016-04-21 22:42:39,2
6,2,d,u6,2016-04-24 01:29:27,3


In [14]:
df_case['Timestamp'] = pd.to_datetime(df['Timestamp'])
df_case = df_case.pivot(index='Case_ID',columns='Count', values='Timestamp')
df_case['Max'] = df_case.apply(max, axis=1)
df_case['Case Duration'] = df_case['Max'] - df_case[1]

df_case

Count,1,2,3,4,5,6,7,Max,Case Duration
Case_ID,,,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36,2016-04-26 10:00:36,11 days 01:19:08
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18,2016-05-01 18:32:18,12 days 21:52:04
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53,2016-05-02 11:06:53,5 days 09:21:46
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28,2016-05-17 14:02:28,15 days 05:23:54
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT,2016-05-11 05:04:34,1 days 21:47:22
6,2016-05-08 07:01:29,2016-05-12 10:39:53,2016-05-14 06:07:39,2016-05-15 21:34:32,2016-05-18 03:47:00,2016-05-18 12:54:53,2016-05-20 01:23:09,2016-05-20 01:23:09,11 days 18:21:40
7,2016-05-18 21:19:05,2016-05-21 10:50:07,2016-05-22 09:40:30,NaT,NaT,NaT,NaT,2016-05-22 09:40:30,3 days 12:21:25
8,2016-05-21 18:49:58,2016-05-22 08:57:32,2016-05-23 23:04:51,2016-05-25 08:31:48,2016-05-28 08:43:24,2016-05-29 20:11:19,2016-06-01 08:27:22,2016-06-01 08:27:22,10 days 13:37:24
9,2016-05-29 16:46:13,2016-05-30 02:05:00,2016-05-31 02:34:15,2016-06-03 06:04:30,2016-06-04 02:42:51,2016-06-04 22:05:31,2016-06-08 23:33:40,2016-06-08 23:33:40,10 days 06:47:27


In [17]:
max_length = df_sort['Count'].max()

In [20]:
df_case['Case Length'] = max_length-df_case.isnull().sum(axis=1)

In [21]:
df_case

Count,1,2,3,4,5,6,7,Max,Case Duration,Case Length
Case_ID,,,,,,,,,,
1,2016-04-15 08:41:28,2016-04-18 12:55:01,2016-04-19 07:22:59,2016-04-23 15:06:58,2016-04-24 19:18:32,2016-04-25 14:56:42,2016-04-26 10:00:36,2016-04-26 10:00:36,11 days 01:19:08,7
2,2016-04-18 20:40:14,2016-04-21 22:42:39,2016-04-24 01:29:27,2016-04-25 22:36:27,2016-04-27 16:12:28,2016-04-28 15:00:35,2016-05-01 18:32:18,2016-05-01 18:32:18,12 days 21:52:04,7
3,2016-04-27 01:45:07,2016-04-27 21:50:32,2016-04-29 00:12:15,2016-04-29 16:24:46,2016-04-30 22:57:03,2016-05-02 01:33:30,2016-05-02 11:06:53,2016-05-02 11:06:53,5 days 09:21:46,7
4,2016-05-02 08:38:34,2016-05-06 00:50:31,2016-05-06 17:56:11,2016-05-13 10:34:23,2016-05-13 13:56:10,2016-05-14 23:42:03,2016-05-17 14:02:28,2016-05-17 14:02:28,15 days 05:23:54,7
5,2016-05-09 07:17:12,2016-05-10 06:29:42,2016-05-11 05:04:34,NaT,NaT,NaT,NaT,2016-05-11 05:04:34,1 days 21:47:22,3
6,2016-05-08 07:01:29,2016-05-12 10:39:53,2016-05-14 06:07:39,2016-05-15 21:34:32,2016-05-18 03:47:00,2016-05-18 12:54:53,2016-05-20 01:23:09,2016-05-20 01:23:09,11 days 18:21:40,7
7,2016-05-18 21:19:05,2016-05-21 10:50:07,2016-05-22 09:40:30,NaT,NaT,NaT,NaT,2016-05-22 09:40:30,3 days 12:21:25,3
8,2016-05-21 18:49:58,2016-05-22 08:57:32,2016-05-23 23:04:51,2016-05-25 08:31:48,2016-05-28 08:43:24,2016-05-29 20:11:19,2016-06-01 08:27:22,2016-06-01 08:27:22,10 days 13:37:24,7
9,2016-05-29 16:46:13,2016-05-30 02:05:00,2016-05-31 02:34:15,2016-06-03 06:04:30,2016-06-04 02:42:51,2016-06-04 22:05:31,2016-06-08 23:33:40,2016-06-08 23:33:40,10 days 06:47:27,7


In [23]:
df_columns = df_sort['Count'].unique()
df_columns

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [24]:
df_case = df_case.drop(df_columns, axis=1)
df_case

Count,Max,Case Duration,Case Length
Case_ID,,,
1,2016-04-26 10:00:36,11 days 01:19:08,7
2,2016-05-01 18:32:18,12 days 21:52:04,7
3,2016-05-02 11:06:53,5 days 09:21:46,7
4,2016-05-17 14:02:28,15 days 05:23:54,7
5,2016-05-11 05:04:34,1 days 21:47:22,3
6,2016-05-20 01:23:09,11 days 18:21:40,7
7,2016-05-22 09:40:30,3 days 12:21:25,3
8,2016-06-01 08:27:22,10 days 13:37:24,7
9,2016-06-08 23:33:40,10 days 06:47:27,7


In [31]:
df_pivot_3 = df_sort.copy()
df_pivot_3 = df_pivot_3.pivot(index='Case_ID', columns='Count', values='User')
df_pivot_3

Count,1,2,3,4,5,6,7
Case_ID,,,,,,,
1,u2,u3,u5,u7,u7,u5,u2
2,u2,u4,u6,u5,u7,u7,u1
3,u2,u3,u5,u5,u8,u8,u2
4,u1,u3,u5,u5,u7,u8,u1
5,u1,u3,u2,NaN,NaN,NaN,NaN
6,u2,u4,u5,u6,u7,u7,u1
7,u1,u3,u1,NaN,NaN,NaN,NaN
8,u2,u4,u6,u5,u7,u7,u1
9,u1,u3,u5,u5,u7,u8,u1


In [32]:
def last_activity(a):
    if a.last_valid_index() is None:
        return np.nan
    else:
        return a[a.last_valid_index()]

In [33]:
df_pivot_3['Last User'] = df_pivot_3.apply(last_activity, axis=1)
df_pivot_3['First User'] = df_pivot_3[1]
df_pivot_3 = df_pivot_3.drop(df_columns,axis=1)
df_pivot_3

Count,Last User,First User
Case_ID,,
1,u2,u2
2,u1,u2
3,u2,u2
4,u1,u1
5,u2,u1
6,u1,u2
7,u1,u1
8,u1,u2
9,u1,u1


In [34]:
df_case = df_case.drop(['Max'],axis=1)

In [35]:
df_pivot_4 = df_pivot.copy()
df_pivot_4 = df_pivot_4.reset_index()
df_pivot_4 = df_pivot_4[['Case_ID','Trace']]
df_pivot_4

Count,Case_ID,Trace
0,1,"u2,u3,u5,u7,u7,u5,u2"
1,2,"u2,u4,u6,u5,u7,u7,u1"
2,3,"u2,u3,u5,u5,u8,u8,u2"
3,4,"u1,u3,u5,u5,u7,u8,u1"
4,5,"u1,u3,u2"
5,6,"u2,u4,u5,u6,u7,u7,u1"
6,7,"u1,u3,u1"
7,8,"u2,u4,u6,u5,u7,u7,u1"
8,9,"u1,u3,u5,u5,u7,u8,u1"
9,10,"u2,u4,u1"


In [36]:
df_pivot_5 = df_pivot_3.merge(df_pivot_4, left_on='Case_ID', right_on='Case_ID', how='inner')

In [37]:
df_case_2 = df_pivot_5.merge(df_case, left_on='Case_ID', right_on='Case_ID', how='inner')
df_case_2.head()

Count,Case_ID,Last User,First User,Trace,Case Duration,Case Length
0,1,u2,u2,"u2,u3,u5,u7,u7,u5,u2",11 days 01:19:08,7
1,2,u1,u2,"u2,u4,u6,u5,u7,u7,u1",12 days 21:52:04,7
2,3,u2,u2,"u2,u3,u5,u5,u8,u8,u2",5 days 09:21:46,7
3,4,u1,u1,"u1,u3,u5,u5,u7,u8,u1",15 days 05:23:54,7
4,5,u2,u1,"u1,u3,u2",1 days 21:47:22,3


In [38]:
df_case_3 = df_case_2.merge(df_pivot_2, left_on='Trace', right_on='Trace', how='outer').sort_values(by='Case_ID')
df_case_3 = df_case_3[['Case_ID','Case Duration','Case Length','Variants','First User','Last User']]
df_case_3['Case Duration'] = df_case_3['Case Duration'].apply(lambda x: round(x.total_seconds(),0))
df_case_3.head()

Count,Case_ID,Case Duration,Case Length,Variants,First User,Last User
0,1,955148.0,7,83,u2,u2
24,2,1115524.0,7,304,u2,u1
27,3,465706.0,7,66,u2,u2
56,4,1315434.0,7,48,u1,u1
90,5,164842.0,3,4,u1,u2


In [39]:
df_case_3.to_csv('Case_2.csv', index=False)

In [40]:
#DF NODE
df_node = df.copy()
df_node = df_node[['Case_ID','Activity','Timestamp','User']]
df_node = df_node.sort_values(by=['Case_ID','Timestamp','Activity'],ascending=True).reset_index(drop=True)

df_node

,Case_ID,Activity,Timestamp,User
0,1,a,2016-04-15 08:41:28,u2
1,1,b,2016-04-18 12:55:01,u3
2,1,d,2016-04-19 07:22:59,u5
3,1,e,2016-04-23 15:06:58,u7
4,1,f,2016-04-24 19:18:32,u7
5,1,g,2016-04-25 14:56:42,u5
6,1,h,2016-04-26 10:00:36,u2
7,2,a,2016-04-18 20:40:14,u2
8,2,b,2016-04-21 22:42:39,u4
9,2,d,2016-04-24 01:29:27,u6


In [45]:
sort1 = df_node.iloc[:-1,:]
sort2 = df_node.iloc[1:,:].reset_index(drop=True)
sort = pd.concat([sort1,sort2],axis=1)
sort.columns = ['Case_ID','Activity1','Timestamp1','Source','Case_ID2','Activity2','Timestamp2','Target']
sort['Comparison'] = sort.apply(lambda x: True if x['Case_ID'] == x['Case_ID2'] else False,axis=1)
sort = sort[sort['Comparison'] == True]
sort['Count'] = sort.groupby('Case_ID').cumcount() + 1
sort = sort.sort_values(by=['Case_ID','Count'], ascending=[True,False])
sort.reset_index(drop=True)
sort['Count_Reverse'] = sort.groupby('Case_ID').cumcount()+1
sort = sort.sort_values(by=['Case_ID', 'Count'], ascending=[True,False]).reset_index(drop=True)
sort['Start_Connect'] = sort['Count'].apply(lambda x: 'Start' if x==1 else '')
sort['End_Connect'] = sort['Count_Reverse'].apply(lambda x: 'End' if x==1 else '')

sort['Timestamp1'] = pd.to_datetime(sort['Timestamp1'])
sort['Timestamp2'] = pd.to_datetime(sort['Timestamp2'])
sort['Duration'] = sort['Timestamp1'] - sort['Timestamp2']
sort['Duration'] = sort['Duration'].apply(lambda x: round(x.total_seconds(),0))

sort

,Case_ID,Activity1,Timestamp1,Source,Case_ID2,Activity2,Timestamp2,Target,Comparison,Count,Count_Reverse,Start_Connect,End_Connect,Duration
0,1,g,2016-04-25 14:56:42,u5,1,h,2016-04-26 10:00:36,u2,True,6,1,,End,-68634.0
1,1,f,2016-04-24 19:18:32,u7,1,g,2016-04-25 14:56:42,u5,True,5,2,,,-70690.0
2,1,e,2016-04-23 15:06:58,u7,1,f,2016-04-24 19:18:32,u7,True,4,3,,,-101494.0
3,1,d,2016-04-19 07:22:59,u5,1,e,2016-04-23 15:06:58,u7,True,3,4,,,-373439.0
4,1,b,2016-04-18 12:55:01,u3,1,d,2016-04-19 07:22:59,u5,True,2,5,,,-66478.0
5,1,a,2016-04-15 08:41:28,u2,1,b,2016-04-18 12:55:01,u3,True,1,6,Start,,-274413.0
6,2,f,2016-04-28 15:00:35,u7,2,h,2016-05-01 18:32:18,u1,True,6,1,,End,-271903.0
7,2,e,2016-04-27 16:12:28,u7,2,f,2016-04-28 15:00:35,u7,True,5,2,,,-82087.0
8,2,g,2016-04-25 22:36:27,u5,2,e,2016-04-27 16:12:28,u7,True,4,3,,,-149761.0
9,2,d,2016-04-24 01:29:27,u6,2,g,2016-04-25 22:36:27,u5,True,3,4,,,-162420.0


In [46]:
sort.to_csv('Node_2.csv', index=False)